In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!conda install -c pyviz --yes --prefix {sys.prefix} geoviews, hvplot, firefox, geckodriver, foilium, selenium

# ISMN + Sentinel Data Dashboard

In [ ]:
# Import modules
from glob import glob
import datetime as dt
import geopandas as gpd
import pandas as pd
import holoviews as hv
import hvplot.pandas
import panel as pn
# Initializes the pyviz notebook extension to allow plotting with bokeh and enable comms
pn.extension()

## Data and Paths

In [ ]:
# Files with data
paths = glob('C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/sentinel_ismn_data/*',)
paths.sort()

# Create Station id linked with filepath
ids = [x.split('\\')[-1].split('_')[0] for x in paths]
files = dict(zip(ids,paths))

In [ ]:
# Load all files into one Dataframe
gdf = gpd.tools.util.pd.concat(map(gpd.read_file, paths), ignore_index=True)
gdf.date = gdf.date.astype('datetime64[ns]')

## Widgets & Dashboard

In [ ]:
#Options for Widgets
years = gdf.date.dt.year.unique()
months = gdf.date.dt.month.unique()
ids = list(gdf.ismn_id.unique())
variables = list(gdf.columns)

# Widgets
id_selector = pn.widgets.Select(name='ISMN Station ID', options=ids, value=1018)
year_slider = pn.widgets.IntSlider(name = 'Year', start=int(years.min()), end=int(years.max()), value=2015)
month_range = pn.widgets.IntRangeSlider(name = 'Months', start=int(months.min()), end=int(months.max()), value=(3,10))
column_picker_x = pn.widgets.Select(name = 'X Axes', options=variables, value = 'soil_moisture')
column_picker_y = pn.widgets.Select(name = 'Y Axes', options=variables, value = 'VV')

@pn.depends(id_selector, year_slider, month_range,column_picker_x,column_picker_y)
def plot_data(id_selector, year_slider, month_range,column_picker_x,column_picker_y):
    d1 = gdf[(gdf['ismn_id'] == id_selector) & (gdf['date'].dt.year == year_slider)& (gdf['date'].dt.month.isin(range(month_range[0], month_range[1])))]
    scatter = d1.hvplot.scatter(x=column_picker_x, y=column_picker_y)
    #d2 = d1.drop
    #reg_line = hv.Slope.from_scatter(scatter)
    return scatter

title = 'Sentinel VV vs. soil moisture (ismn station)'
header_box = pn.WidgetBox(title,id_selector, year_slider, month_range,column_picker_x, column_picker_y, pn.layout.Spacer(margin=200), width=300, height=1000, align="center")# Plot Box
dashboard = pn.Row(header_box, plot_data)
dashboard

## Dashboard

In [ ]:
title = 'Sentinel VV vs. soil moisture (ismn station)'
header_box = pn.WidgetBox(title,id_selector, year_slider,month_range, pn.layout.Spacer(margin=200), width=300, height=1000, align="center")# Plot Box
dashboard = pn.Row(header_box, plot_data)
dashboard

In [ ]:
#Options for Widgets
years = gdf.date.dt.year.unique()
months = gdf.date.dt.month.unique()
ids = list(gdf.ismn_id.unique())
variables = list(gdf.columns)

# Widgets
id_selector = pn.widgets.Select(name='ISMN Station ID', options=ids)
year_slider = pn.widgets.IntSlider(name = 'Year', start=int(years.min()), end=int(years.max()))
month_range = pn.widgets.IntRangeSlider(name = 'Months', start=int(months.min()), end=int(months.max()))
column_picker_x = pn.widgets.Select(name = 'X Axes', value = 'soil_moisture', options=variables)
column_picker_y = pn.widgets.Select(name = 'Y Axes', value = 'VV', options=variables)

def sel_gdf(gdf, idx, year, month_min, month_max, x, y):
    gdf = gdf[(gdf['ismn_id'] == idx) & (gdf['date'].dt.year == year)& (gdf['date'].dt.month == range(month_min, month_max))]
    return gdf.hvplot.scatter(x = x, y = y)

pn.Column(id_selector, year_slider, month_range, column_picker_x, column_picker_y)
dmap = hv.DynamicMap(sel_gdf, kdims=['idx', 'year', 'month_min','month_max', 'x', 'y'])
dmap[gdf, id_selector.value, year_slider.value, month_range.value[0], month_range.value[1], column_picker_x.value, column_picker_y.value]

In [ ]:
files = glob('C://Users/USER/Desktop/Master_Irrigation/03_GIS/ground_trouth/sentinel_ismn_data/*',)



def split_s1(file):
    """
    Arguments:
    """
    
    # Import modules
    import geopandas as gpd
    
    # Read file
    gdf = gpd.read_file(file)
    gdf['date'] = gdf.date.astype('datetime64[ns]')
    # Split into 4 groups of equal aquisition types
    asA = gdf[(gdf['orbit'] == 'ASCENDING') & (gdf['platform'] == 'A')].dropna(how='any', subset=['soil_moisture'])
    dsA = gdf[(gdf['orbit'] == 'DESCENDING') & (gdf['platform'] == 'A')].dropna(how='any', subset=['soil_moisture'])
    asB = gdf[(gdf['orbit'] == 'ASCENDING') & (gdf['platform'] == 'B')].dropna(how='any', subset=['soil_moisture'])
    dsB = gdf[(gdf['orbit'] == 'DESCENDING') & (gdf['platform'] == 'B')].dropna(how='any', subset=['soil_moisture'])
    
    return asA, dsB, asB, dsB

asA, dsB, asB, dsB = split_s1(files[0])

import hvplot.pandas  # noqa
asA[['VV', 'soil_moisture', 'date']].hvplot.scatter(x='VV', y='soil_moisture', by='date.year')

In [ ]:
import geopandas as gpd
import pandas as pd

## Load FeatureCollection from 07_Sentinel_1_grd notebook

In [ ]:
def load_full_gdf(path):
    gdf = gpd.read_file(path)
    gdf = gdf.astype({'sentinel': 'string', 'orbit': 'string', 'identifier': 'string'})
    gdf['date'] = pd.to_datetime(gdf['date'])
    return pd.DataFrame(gdf)

In [ ]:
df_raw = load_full_gdf(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/whr_s1_raw_full.geojson')
df_raw = df_raw[df_raw['date'].dt.year >= 2016]

df_ndvi15 = load_full_gdf(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/whr_s1_ndvi15_full.geojson')
df_ndvi30 = load_full_gdf(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/whr_s1_ndvi30_full.geojson')

In [ ]:
def compute_swi(df):
    #5th and 95th percentile represting dry and wet state
    vvdry = df.VV.quantile(0.05, 'linear')
    vvwet = df.VV.quantile(0.95, 'linear')
    
    swi = list()
    for row in df.itertuples():
        vv0 = row.VV
        swi.append(((vv0 - vvdry) / (vvwet - vvdry)) * 100)
    
    return swi

In [ ]:
df_ndvi15['swi'] = compute_swi(df_ndvi15)

In [ ]:
#Create 4 Groups of similiar conditions and sort time series afte patch numbers
ms1 = gdf[(gdf['sentinel'] == 'A') & (gdf['orbit'] == 'descending')].sort_values(['patch','date'])
ms2 = gdf[(gdf['sentinel'] == 'B') & (gdf['orbit'] == 'descending')].sort_values(['patch','date'])
as1 = gdf[(gdf['sentinel'] == 'A') & (gdf['orbit'] == 'ascending')].sort_values(['patch','date'])
as2 = gdf[(gdf['sentinel'] == 'B') & (gdf['orbit'] == 'ascending')].sort_values(['patch','date'])

# Visualize with matplotlib

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import dates
import matplotlib.dates as mdates
import datetime as dt

In [ ]:
#Plot with matplotlib and pandas
def filter_data(df,year,patch,orbit,sentinel):
    return df[(df['sentinel'] == sentinel) & (df['orbit'] == orbit) & (df['patch'] == patch) & (df['date'].dt.year == year)].sort_values('date')

In [ ]:
df = filter_data(df_ndvi15,2018,4,'descending','A')
df1 = filter_data(df_ndvi30,2018,4,'ascending','A')
df2 = filter_data(df_ndvi30,2018,4,'ascending','B')
df3 = filter_data(df_ndvi30,2018,4,'descending','B')

In [ ]:
fig1 = plt.Figure(figsize=(19,9))  # Make a figure

# Bar charts 
ax1 = fig1.add_subplot()         
ax1.bar('date', 'precipitation', data=df, color='black')   
ax1.set_ylabel('precipitation sum between two measurements (mm)', color='black')
ax1.set_xticks(df.date.to_list())
ax1.set_xticks(df.date.to_list(), minor=True)

# Line Chart second y axis
ax2 = ax1.twinx()               
ax2.plot('date', 'swi', data=df, color='red', marker='o', linewidth=5)
#ax2.plot('date', 'VV_change', data=df1, color='darkred', marker='o', linewidth=5)
#ax2.plot('date', 'VV_change', data=df2, color='blue', marker='o', linewidth=5)
#ax2.plot('date', 'SWI', data=df3, color='darkblue', marker='o', linewidth=5)
ax2.set_ylabel('Backscatter coefficient dB', color='black')

# Format X axes
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax1.grid(True, alpha=1, which='both')

ax1.set_xlim([dt.datetime(2018,3,1),dt.datetime(2018,11,1)], auto=True)
display(fig1)

In [ ]:
df.plot(x='date', y='swi')

# Visualize with Hvplot (holoviews)

In [ ]:
import hvplot
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [ ]:
line_raw = df_raw.hvplot.line(x='date', y='VV', rot=90, groupby=['date.year','date.month', 'patch', 'sentinel', 'orbit'])
line_15 = df_ndvi15.hvplot.line(x='date', y='VV', rot=90, groupby=['date.year','date.month', 'patch', 'sentinel', 'orbit'])
line_30 = df_ndvi30.hvplot.line(x='date', y='VV', rot=90, groupby=['date.year','date.month', 'patch', 'sentinel', 'orbit'])

(line_raw * line_15 * line_30).opts(legend_position='right')

In [ ]:
scatter = gdf_raw.hvplot.scatter(x='date', y='VV_change',rot=90, groupby=['date.year', 'patch', 'sentinel', 'orbit'])
hist = gdf_raw.hvplot.hist('precipitation', rot=90, groupby=['date.year', 'patch', 'sentinel', 'orbit'])
ls1 = hv.link_selections.instance()
scatter + hist

In [ ]:
#irrigation detection 
irrigation = []
for i, row in zip(tqdm(range(len(gdf))),gdf.itertuples()):
    pp_sum = row.precipitation
    VV_change = row.VV_change
    if VV_change >= 1:
        irrigation.append('')

In [ ]:
df.dtypes